In [1]:
import numpy as np
import csv
import json
import os 

In [2]:
import config



In [5]:
class Data:
    def __init__(self):
        self.verbose = False
        self.parsed = {}
        
    def process_team(self, team_name: str, team_names: list, verbose = False):
        self.verbose = verbose
        
        if (self.verbose):
            print("==============\nTEAM: {}\n---".format(team_name))

        for user_name in team_names:
            path = config.path(user_name)
            if (self.verbose):
                print("\t +++ {} +++ \n\tDATA: {} \n---".format(user_name, path))
                
            self.parsed[team_name] = self.parse_user(user_name, path)
            
            if (self.verbose):
                print("\t PARSED")
        
        self.verbose = False
        
    def parse_user(self, user_name, path):
        for _, dir in config.dir_names().items():
            if (self.verbose):
                print("\t\t--- {} ---".format(dir))
            full_path = os.path.join(path, dir)
            for filename in os.listdir(full_path): 
                print(filename)

In [6]:
data = Data()
data.process_team("SOMHunter", ["sh-patrik", "sh-vit"], verbose=True)


TEAM: SOMHunter
---
	 +++ sh-patrik +++ 
	DATA: ./data/sh-patrik 
---
		--- actions ---
user-actions.21-06-2021T12-48-05.admin.json
user-actions.23-06-2021T11-00-07.admin.json
		--- results ---
results.21-06-2021T12-48-05.admin.json
results.23-06-2021T11-00-07.admin.json
		--- eval-server-requests ---
1624272485332__login.json
1624275276925__interaction.json
1624275277442__result.json
1624275287298__interaction.json
1624275290885__result.json
1624275301422__interaction.json
1624275321078__interaction.json
1624275321509__result.json
1624275334140__interaction.json
1624275341144__result.json
1624275347496__interaction.json
1624275354474__result.json
1624275360150__interaction.json
1624275371914__interaction.json
1624275372516__result.json
1624275388335__interaction.json
1624275399252__interaction.json
1624275401874__result.json
1624275420012__interaction.json
1624275451562__interaction.json
1624275451964__result.json
1624275463157__interaction.json
1624275463517__result.json
162427547588

FileNotFoundError: [WinError 3] The system cannot find the path specified: './data/sh-vit\\actions'